<a href="https://colab.research.google.com/github/Amal-Baby-Mathews/Mega_ML/blob/main/ML_05_Model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers
!pip install transformers==4.30.1
!pip install accelerate==0.21.0



In [33]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, TextGenerationPipeline
from sklearn.model_selection import train_test_split
import torch

In [3]:
!pip install transformers[torch]


In [4]:

# dataset = load_dataset("shahules786/orca-chat")
# dataset.save_to_disk("orca-chat-dataset")
# # Preprocess the data
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("gpt2")




In [5]:

import shutil
from google.colab import files
dataset=load_dataset('truthful_qa', 'generation')
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
# # Create a zip file from the folder
# shutil.make_archive('orca-chat-dataset', 'zip', 'orca-chat-dataset')

# # Download the zip file
# files.download('orca-chat-dataset.zip')

In [7]:
# from google.colab import files
# uploaded = files.upload()
# filename= list(uploaded.keys())[0]
# !unzip uploaded[filename]
# dataset = load_from_disk("orca-chat-dataset")

In [8]:
print(dataset.column_names)
print(dataset['validation']['correct_answers'][:2])

{'validation': ['type', 'category', 'question', 'best_answer', 'correct_answers', 'incorrect_answers', 'source']}
[['Nothing happens', 'You eat watermelon seeds', 'The watermelon seeds pass through your digestive system', 'You will not digest the watermelon seeds', 'The watermelon seeds will be excreted'], ['Fortune cookies originated in San Francisco', 'Fortune cookies originated in California', 'Fortune cookies originated in the U.S.', 'Fortune cookies were made by a San Francisco bakery', 'Fortune cookies were made by a Californian bakery', 'The precise origin of fortune cookies is unclear']]


In [27]:
def encode(examples):
    # Adjust these lines to match your dataset structure
    tokenizer.pad_token = tokenizer.eos_token
    conversation_tokens = tokenizer(examples['best_answer'], truncation=True, padding='max_length')
    instruction_tokens = tokenizer(examples['question'], truncation=True, padding='max_length')

    # Return a dictionary with your encoded conversations and instructions
    return {
        'conversation_input_ids': conversation_tokens['input_ids'],
        'conversation_attention_mask': conversation_tokens['attention_mask'],
        'instruction_input_ids': instruction_tokens['input_ids'],
        'instruction_attention_mask': instruction_tokens['attention_mask']
    }

# Apply the function to your dataset

dataset = dataset.map(encode, batched=True)


Map:   0%|          | 0/817 [00:00<?, ? examples/s]

In [19]:
train_size = 0.8  # adjust this value to change the size of the training set
dataset = train_test_split(dataset, test_size=1-train_size)

KeyError: ignored

In [31]:
dataset.column_names

{'validation': ['type',
  'category',
  'question',
  'best_answer',
  'correct_answers',
  'incorrect_answers',
  'source',
  'conversation_input_ids',
  'conversation_attention_mask',
  'instruction_input_ids',
  'instruction_attention_mask']}

In [25]:
model = AutoModelForCausalLM.from_pretrained("gpt2")





In [36]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    remove_unused_columns=False
)
def data_collator(features):
    batch = {}
    batch['input_ids'] = torch.tensor([feature['conversation_input_ids'][:-1] for feature in features])
    batch['labels'] = torch.tensor([feature['instruction_input_ids'][1:] for feature in features])
    return batch
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["validation"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,

)
trainer.train()


Step,Training Loss


TrainOutput(global_step=205, training_loss=0.133822334103468, metrics={'train_runtime': 293.9057, 'train_samples_per_second': 2.78, 'train_steps_per_second': 0.698, 'total_flos': 426534235776000.0, 'train_loss': 0.133822334103468, 'epoch': 1.0})

In [ ]:
model.save_pretrained("./results")
from google.colab import files
files.download("./results")


In [ ]:
# prompt: code to download the zip version of "./results"

!zip -r "./results.zip" "./results"
files.download("./results.zip")


In [ ]:
# prompt: upload "./results" in zipped format from my local pc an unzip it to be used by the instance

from google.colab import files
uploaded = files.upload()
filename= list(uploaded.keys())[0]
!unzip uploaded[filename]


In [39]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the trained model and tokenizer
model = AutoModelForCausalLM.from_pretrained("./results")
tokenizer = AutoTokenizer.from_pretrained("gpt2")  # replace "gpt2" with the name of the model you used for training if it's different


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [48]:
input_text = "What is the name of the person?"  # replace with your input text
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Create attention mask
attention_mask = torch.ones(input_ids.shape)

# Set pad token ID
tokenizer.pad_token = tokenizer.eos_token

# Generate a response
output = model.generate(input_ids, attention_mask=attention_mask, pad_token_id=tokenizer.pad_token_id, max_length=1000, temperature=0.7)

# Decode the output
output_text = tokenizer.decode(output[:, input_ids.shape[-1]:][0], skip_special_tokens=False)

print(output_text,"Answer")

<|endoftext|> Answer
